首先 import 必要的模块

In [1]:
import lightgbm as lgbm
#from lgbm.sklearn import LGBMClassifier
import xlearn as xl
import pandas as pd 
import numpy as np
import time

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss     #采用logloss作为评价指标
from pandas import DataFrame

from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline
from libffm_data_generate import *

## 1、 数据输入

In [2]:
train_name_1='100_0'
train_name_2='100_1'
valid_name_1='100_2'
valid_name_2='100_3'

In [159]:
#train_data
train_1 = pd.read_csv("/home/jupyter/data_final_29/100w/"+train_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_1.drop(['id','hour','user_id','hour_days'],axis=1,inplace=True)
print(train_1.shape)
#validation_data
valid_1 = pd.read_csv("/home/jupyter/data_final_29/100w/"+valid_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_1.drop(['id','hour','user_id','hour_days'],axis=1,inplace=True)
print(valid_1.shape)

(999936, 25)
(999188, 25)


In [160]:
#train_data
train_2 = pd.read_csv("/home/jupyter/data_final_29/100w/"+train_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_2.drop(['id','hour','user_id','hour_days'],axis=1,inplace=True)
print(train_2.shape)
valid_2 = pd.read_csv("/home/jupyter/data_final_29/100w/"+valid_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_2.drop(['id','hour','user_id','hour_days'],axis=1,inplace=True)
print(valid_2.shape)

(999936, 25)
(994263, 25)


In [3]:
#test_data
test = pd.read_csv("/home/jupyter/data_final_29/test",index_col=0).reset_index(drop=True)   #拿200万条数据训练
test.drop(['id','hour','user_id','hour_days'],axis=1,inplace=True)
test.shape

(4218938, 25)

In [162]:
#first
y_train_1 = train_1['click']
x_gbm_train_1=train_1[['user_hour_C14','user_hour_C17','user_hour_app_id']]
x_ffm_train_1=train_1.drop(['user_hour_C14','user_hour_C17','user_hour_app_id',"click"], axis=1)

y_valid_1 = valid_1['click']
x_gbm_valid_1=valid_1[['user_hour_C14','user_hour_C17','user_hour_app_id']]
x_ffm_valid_1=valid_1.drop(['user_hour_C14','user_hour_C17','user_hour_app_id',"click"], axis=1)

In [163]:
#second
y_train_2 = train_2['click']
x_gbm_train_2=train_2[['user_hour_C14','user_hour_C17','user_hour_app_id']]
x_ffm_train_2=train_2.drop(['user_hour_C14','user_hour_C17','user_hour_app_id',"click"], axis=1)

y_valid_2 = valid_2['click']
x_gbm_valid_2=valid_2[['user_hour_C14','user_hour_C17','user_hour_app_id']]
x_ffm_valid_2=valid_2.drop(['user_hour_C14','user_hour_C17','user_hour_app_id',"click"], axis=1)

In [4]:
y_test = test['click']
x_gbm_test=test[['user_hour_C14','user_hour_C17','user_hour_app_id']]
x_ffm_test=test.drop(['user_hour_C14','user_hour_C17','user_hour_app_id',"click"], axis=1)

In [5]:
del train_1
del train_2
del valid_1
del valid_2
del test

## 2. 将连续性特征输入lightgbm生成编码后的特征

In [177]:
start=time.time()
# 定义模型
lgb1 = lgbm.LGBMRegressor(
          num_boost_round=20,
          boosting='gbdt', 
          objective='binary', 
          num_threads=6, 
          #default=OpenMP_default, type=int, alias=num_thread, nthread LightGBM 的线程数
#          'silent': True,
          learning_rate=0.1, 
          num_leaves=10, 
          #num_leaves：叶子数
#num_leaves, default=31, type=int, alias=num_leaf一棵树上的叶子数
          max_depth= 3,
          max_bi= 256, 
          subsample_for_bin= 200000,#默认200000，用来构建直方图数据数量
          subsample= 0.8, #默认1
          #类似于feature_fraction,它将在不进行重采样的情况下
          #随机选择部分数据可以用来加速训练可以用来处理过拟合Not
          subsample_freq= 1, 
          colsample_bytree= 0.8,       
          reg_alpha= 0.8,         
          reg_lambda= 0.25, 
          min_split_gain=0.0, 
          min_child_weight= 1,        
          min_child_samples= 200, 
          min_data_in_leaf=50, 
          #default=20, type=int, alias=min_data_per_leaf , min_data, 
          #min_child_samples一个叶子上数据的最小数量. 可以用来处理过拟合.
        
        # 'categorical_feature':3
         # 'categorical_feature':3:26
         )
# 训练学习
model=lgb1.fit(x_gbm_train_1, y_train_1)
end=time.time()
print(end-start)

/home/qiaoliang/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.481795072555542


In [178]:
# train上logloss评测
y_pred_train = lgb1.predict(x_gbm_train_1)
logloss = log_loss(y_train_1, y_pred_train)
print ("logloss of train :",logloss )

# valid上logloss评测
y_pred_valid = lgb1.predict(x_gbm_valid_1)
logloss = log_loss(y_valid_1, y_pred_valid)
print ("logloss of valid :",logloss )

# test上logloss评测
y_pred_test = lgb1.predict(x_gbm_test)
logloss = log_loss(y_test, y_pred_test)
print ("logloss of test :",logloss )

logloss of train : 0.44975218594366606
logloss of valid : 0.4496632713427818
logloss of test : 0.44907588321989766


##### 生成每颗树最大叶子节点的字典，需手动设置n_tree和num_leaves

In [181]:
dict_gbm={}
n_tree=20
num_leaves=10
for i in range(n_tree):
    dict_gbm[i]=num_leaves

##### 第1组数据，将经过gbm处理的连续特征按叶子节点输出

In [180]:
x_train_leaves = lgb1.predict(x_gbm_train_1, pred_leaf=True)    
x_valid_leaves = lgb1.predict(x_gbm_valid_1, pred_leaf=True) 
x_test_leaves = lgb1.predict(x_gbm_test, pred_leaf=True) 
print(x_train_leaves.shape)
print(x_valid_leaves.shape)
print(x_test_leaves.shape)

(999936, 20)
(999188, 20)
(4218938, 20)


##### 将第2组数据，将经过gbm处理的连续特征按叶子节点输出

In [46]:
x_train_leaves = lgb1.predict(x_gbm_train_2, pred_leaf=True)    
x_valid_leaves = lgb1.predict(x_gbm_valid_2, pred_leaf=True) 
print(x_train_leaves.shape)
print(x_valid_leaves.shape)

(1000, 30)
(1000, 30)


## 3、将原始特征与经GBM编码后的特征进行整合处理为供FFM的特征

### 3.1 lightgbm输出叶子节点数据转化为libffm或liffm格式数据

In [182]:
#ffm-data
#train
start=time.time()
original_gbm_ffm_data(x_ffm_train_1,y_train_1,dict_gbm,x_train_leaves,save_name=r'/home/jupyter/xlearn/gbm_orig_ffm/ffm_'+train_name_1+'.txt')
end=time.time()
print(end-start)

310.03121614456177


In [183]:
#valid
start=time.time()
original_gbm_ffm_data(x_ffm_valid_1,y_valid_1,dict_gbm,x_valid_leaves,save_name=r'/home/jupyter/xlearn/gbm_orig_ffm/ffm_'+valid_name_1+'.txt')
end=time.time()
print(end-start)

293.6466438770294


In [184]:
#test
start=time.time()
original_gbm_ffm_data(x_ffm_test,y_test,dict_gbm,x_test_leaves,save_name=r'/home/jupyter/xlearn/gbm_orig_ffm/'+'ffm_test.txt')
end=time.time()
print(end-start)

1266.9010841846466


### 4 FFM模型训练

##### 原始数据的ffm模型初始化

In [7]:
# original Training task
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain(r'/home/jupyter/xlearn/gbm_orig_ffm/ffm_'+train_name_1+'.txt')  # Training data
ffm_model.setValidate(r'/home/jupyter/xlearn/gbm_orig_ffm/ffm_'+valid_name_1+'.txt')  # Validation data
#ffm_model.setTest(r'/home/jupyter/xlearn/gbm_orig_ffm/ffm_'+valid_name_1+'.txt')  # test data
ffm_model.setTest(r'/home/jupyter/xlearn/gbm_orig_ffm/'+'ffm_test.txt')  # test data

##### 模型训练

In [186]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0002, 
         'metric':'acc','fold':2,'epoch':40,'stop_window':3}
#ffm_model.setSign() # Convert output to 0-1  scope
ffm_model.setSigmoid()
result=ffm_model.fit(param, r'/home/jupyter/xlearn/gbm_orig_ffm/model.out')
result_cv=ffm_model.cv(param)
end=time.time()
print(end-start)

313.62089800834656


In [187]:
# Prediction task
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/gbm_orig_ffm/model.out', r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt')

y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.4097700772907551


#### FFM模型参数调整-调整隐层K，K还是取4吧

In [188]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':8,'opt':'adagrad','lr':0.2,'lambda':0.0002, 
         'metric':'acc','fold':2,'epoch':40,'stop_window':3}
#ffm_model.setSign() # Convert output to 0-1  scope
ffm_model.setSigmoid()
result=ffm_model.fit(param, r'/home/jupyter/xlearn/gbm_orig_ffm/model.out')
result_cv=ffm_model.cv(param)
end=time.time()
print(end-start)

# Prediction task
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/gbm_orig_ffm/model.out', r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt')
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )
#0.4001,0.4091,0.415

391.52443170547485
logloss of test : 0.41501459668314483


##### lambda调整，0.0001,0.0005

In [189]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':8,'opt':'adagrad','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':40,'stop_window':3}
#ffm_model.setSign() # Convert output to 0-1  scope
ffm_model.setSigmoid()
result=ffm_model.fit(param, r'/home/jupyter/xlearn/gbm_orig_ffm/model.out')
result_cv=ffm_model.cv(param)
end=time.time()
print(end-start)

# Prediction task

374.90474033355713


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [190]:
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/gbm_orig_ffm/model.out', r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt')
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.4089657915496744


In [22]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.000075, 
         'metric':'acc','fold':2,'epoch':40,'stop_window':3}
#ffm_model.setSign() # Convert output to 0-1  scope
ffm_model.setSigmoid()
result=ffm_model.fit(param, r'/home/jupyter/xlearn/gbm_orig_ffm/model.out')
result_cv=ffm_model.cv(param)
end=time.time()
print(end-start)

248.4234766960144


In [27]:
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/gbm_orig_ffm/model.out', r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt')
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.4075080439133808


In [14]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00005, 
         'metric':'acc','fold':2,'epoch':21,'stop_window':3}
#ffm_model.setSign() # Convert output to 0-1  scope
ffm_model.setSigmoid()
result=ffm_model.fit(param, r'/home/jupyter/xlearn/gbm_orig_ffm/model.out')
result_cv=ffm_model.cv(param)
end=time.time()
print(end-start)

154.01739239692688


In [15]:
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/gbm_orig_ffm/model.out', r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt')
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/gbm_orig_ffm/output.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.410874928466404
